<a href="https://colab.research.google.com/github/AtharKharal/Data_science_with_python/blob/master/workflow_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

#Data Reading

In [0]:
# Go to the CSV file in Google Drive, right-click on it and select 
# “Get shareable link”. The link will be copied into your clipboard. 
# Paste this link into following a string variable i.e. link:

link = 'https://drive.google.com/open?id=1SFqRrx4zqqud34_swwrLl5FHm6IFjNQD'  # The shareable link

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


fluff, id = link.split('=')   # Verify that you have everything after '=' by using print(id)

# Finally reading the data into a dataframe

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dummy_file_name.csv')  
df1 = pd.read_csv('dummy_file_name.csv')

In [0]:
target=df1['Survived']
df1=df1.drop(labels='Survived',axis=1)
df1.head()

,Sex,Age,FamilySize,FamilyOneSurvived,FamilyAllDied,FareAdj
0,0,0.2750,0.181818,0,1,7.25000
1,1,0.4750,0.181818,0,0,35.64165
2,1,0.3250,0.090909,0,0,7.92500
3,1,0.4375,0.181818,0,1,26.55000
4,0,0.4375,0.090909,1,0,8.05000


#featuretools

In [0]:
#================== Installing featuretools and its compatibilities   #numpy==1.14.6

#!pip install tornado==4.5.3 featuretools==0.6.1 pandas==0.24.1


In [0]:
import featuretools as ft

#import featuretools.variable_types as vt

In [0]:
df1['df_index']= np.arange(df1.shape[0])
es = ft.EntitySet(id='myEntitySet_id')
es.entity_from_dataframe(dataframe=df1,
                         entity_id='df1_id',
                         index='df_index'
                        )
#es.normalize_entity(base_entity_id='df1_id', new_entity_id='df2_id', index= 'Sex' )

Entityset: myEntitySet_id
  Entities:
    df1_id [Rows: 891, Columns: 7]
  Relationships:
    No relationships

In [0]:
fm, features = ft.dfs(entityset=es, target_entity='df1_id',
                      verbose = True                        
                     )

Built 6 features
Elapsed: 00:03 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [0]:
features

[<Feature: Sex>,
 <Feature: Age>,
 <Feature: FamilySize>,
 <Feature: FamilyOneSurvived>,
 <Feature: FamilyAllDied>,
 <Feature: FareAdj>]

#TPOT

In [0]:
!pip install tpot

In [0]:
import sklearn.model_selection
import tpot

In [0]:
X, y = df1 , target

#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

tpot = tpot.TPOTClassifier(generations=100, population_size=100, verbosity=2, 
                           n_jobs = -1, cv=5, config_dict='TPOT light'
                          )

# many more parameters may be set as follows:
# scoring = 'neg_mean_absolute_error', max_time_mins = 480, n_jobs = -1,
# cv = 5, config_dict='TPOT light'

tpot.fit(X, y)

In [0]:
print(tpot.score(X_test, y_test))
#y_hat = tpot.predict(X_test)

0.8430493273542601


In [0]:
tpot.export("titanic_ppl2.py")

#auto-sklearn

In [0]:
#================== Installing auto-sklearn on colab
#!sudo apt-get install build-essential swig
#!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
#!pip install auto-sklearn

In [0]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

In [0]:
X, y = fm, target
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train )

y_hat = automl.predict(X_test)

print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

[WARNING] [2019-03-12 13:08:51,426:EnsembleBuilder(1):fe25c0fdcc73562ab16c5ca7cb8053f3] No models better than random - using Dummy Score!
[WARNING] [2019-03-12 13:08:51,435:EnsembleBuilder(1):fe25c0fdcc73562ab16c5ca7cb8053f3] No models better than random - using Dummy Score!
[WARNING] [2019-03-12 13:08:53,600:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-03-12 13:08:53,600:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
Accuracy score 0.8071748878923767


In [0]:
automl.get_models_with_weights()